In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import  ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
!pip install torch

  Using cached https://files.pythonhosted.org/packages/60/7a/e881b5abb54db0e6e671ab088d079c57ce54e8a01a3ca443f561ccadb37e/typing_extensions-3.7.4.3-py3-none-any.whl


In [6]:
!pip install torchvision

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [9]:
#MNIST 数据集
#设置训练的批次大小、学习率、及训练代数
batch_size=200
learning_rate=0.001
epochs=20


#下载数据集
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Processing...
Done!


In [10]:
#生成 三个神经网络成，对应感知节分别为第一层100，第二成200，第三层10，即要分类的数目
w1, b1 = torch.randn(100, 784, requires_grad=True),\
         torch.zeros(100, requires_grad=True)
w2, b2 = torch.randn(200, 100, requires_grad=True),\
         torch.zeros(200, requires_grad=True)
w3, b3 = torch.randn(10, 200, requires_grad=True),\
         torch.zeros(10, requires_grad=True)

In [11]:
#定义前向网络计算，每层神经网络输出后增加relu激活函数，确保网络的非线性，实现更好的分类效果
def forward(x):
    x = x@w1.t() + b1
    x = F.relu(x)
    x = x@w2.t() + b2
    x = F.relu(x)
    x = x@w3.t() + b3
    x = F.relu(x)
    return x

In [12]:
#定义优化器，采用SGD随机梯度下降的方式对w1, b1, w2, b2, w3, b3进行优化
optimizer = optim.SGD([w1, b1, w2, b2, w3, b3], lr=learning_rate)
#定义采用交叉熵作为损失函数
criteon = nn.CrossEntropyLoss()

In [13]:
#设置迭代次数
for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        #将数据打平为（批次，高度*宽度），-1代表所有
        data = data.view(-1, 28*28)

        #将数据输入到网络中
        cal_data = forward(data)
        #将计算的数据与目标数据求误差损失
        loss = criteon(cal_data, target)

        #将梯度值初始化为0
        optimizer.zero_grad()
        #pytorch计算梯度值
        loss.backward()
        #更新梯度值
        optimizer.step()
        #每隔25*batcsize(200) = 5000 打印输出结果
        if batch_idx % 25 == 0:
            print('训练代数: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

    #将测试误差及正确率清0
    test_loss = 0
    correct = 0
    #取测试集数据及目标数据
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = forward(data)
        #误差累加
        test_loss += criteon(logits, target).item()
        #取出预测最大值的索引编号，即预测值
        pred = logits.data.argmax(dim=1)
        #统计正确预测的个数
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    #打印输出测试误差及准确率
    print('\n测试集: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

训练代数: 0 [0/60000 (0%)]	Loss: 2010.320435
训练代数: 0 [5000/60000 (8%)]	Loss: 24.145620
训练代数: 0 [10000/60000 (17%)]	Loss: 10.370474
训练代数: 0 [15000/60000 (25%)]	Loss: 4.403249
训练代数: 0 [20000/60000 (33%)]	Loss: 4.524137
训练代数: 0 [25000/60000 (42%)]	Loss: 5.080046
训练代数: 0 [30000/60000 (50%)]	Loss: 2.785926
训练代数: 0 [35000/60000 (58%)]	Loss: 6.190485
训练代数: 0 [40000/60000 (67%)]	Loss: 2.302583
训练代数: 0 [45000/60000 (75%)]	Loss: 4.070419
训练代数: 0 [50000/60000 (83%)]	Loss: 2.279557
训练代数: 0 [55000/60000 (92%)]	Loss: 2.291070

测试集: Average loss: 0.0135, Accuracy: 985/10000 (10%)

训练代数: 1 [0/60000 (0%)]	Loss: 4.840629
训练代数: 1 [5000/60000 (8%)]	Loss: 2.836610
训练代数: 1 [10000/60000 (17%)]	Loss: 2.291070
训练代数: 1 [15000/60000 (25%)]	Loss: 2.302583
训练代数: 1 [20000/60000 (33%)]	Loss: 2.294051
训练代数: 1 [25000/60000 (42%)]	Loss: 2.302583
训练代数: 1 [30000/60000 (50%)]	Loss: 2.302583
训练代数: 1 [35000/60000 (58%)]	Loss: 2.527578
训练代数: 1 [40000/60000 (67%)]	Loss: 2.302583
训练代数: 1 [45000/60000 (75%)]	Loss: 2.302583
训练代数: 1 

训练代数: 14 [25000/60000 (42%)]	Loss: 2.291070
训练代数: 14 [30000/60000 (50%)]	Loss: 2.302583
训练代数: 14 [35000/60000 (58%)]	Loss: 2.302583
训练代数: 14 [40000/60000 (67%)]	Loss: 2.302583
训练代数: 14 [45000/60000 (75%)]	Loss: 2.302583
训练代数: 14 [50000/60000 (83%)]	Loss: 2.302583
训练代数: 14 [55000/60000 (92%)]	Loss: 2.302583

测试集: Average loss: 0.0115, Accuracy: 981/10000 (10%)

训练代数: 15 [0/60000 (0%)]	Loss: 2.302583
训练代数: 15 [5000/60000 (8%)]	Loss: 2.302583
训练代数: 15 [10000/60000 (17%)]	Loss: 2.302583
训练代数: 15 [15000/60000 (25%)]	Loss: 2.302583
训练代数: 15 [20000/60000 (33%)]	Loss: 2.302583
训练代数: 15 [25000/60000 (42%)]	Loss: 2.302583
训练代数: 15 [30000/60000 (50%)]	Loss: 2.302583
训练代数: 15 [35000/60000 (58%)]	Loss: 2.302583
训练代数: 15 [40000/60000 (67%)]	Loss: 2.302583
训练代数: 15 [45000/60000 (75%)]	Loss: 2.302583
训练代数: 15 [50000/60000 (83%)]	Loss: 2.302583
训练代数: 15 [55000/60000 (92%)]	Loss: 2.302583

测试集: Average loss: 0.0115, Accuracy: 981/10000 (10%)

训练代数: 16 [0/60000 (0%)]	Loss: 2.302583
训练代数: 16 [5000/60000 (